In [1]:
import os
path = "/Users/parker.erickson/supreme_court_transcripts/oyez/cases/"
cases = os.listdir(path)

In [2]:
print(len(cases))

16425


In [3]:
import json

validCases = []

for case in cases:
    try:
        caseDetails = json.load(open(path + case))
        if type(caseDetails) == dict:
            if (caseDetails["facts_of_the_case"] is not None and caseDetails["facts_of_the_case"] != '') and (caseDetails["question"] is not None and caseDetails["question"] != '') and (caseDetails["decisions"] is not None and caseDetails["decisions"] != ''):
                validCases.append(caseDetails)
    except:
        pass

In [4]:
len(validCases)

3122

In [5]:
def parseCase(case):
    try:
        cn = case["name"].split("v.")
        plantiff = cn[0].strip()
        defendant = cn[1].strip()
        question = case["question"].strip().strip("<p>").strip("</p>").strip("\n")
        facts = case["facts_of_the_case"].strip().strip("<p>").strip("</p>").strip("\n")

        question = question.replace(plantiff, "plantiff").replace(defendant, "defendant")
        facts = facts.replace(plantiff, "plantiff").replace(defendant, "defendant")

        winningParty = case["decisions"][0]["winning_party"]
        inFavorPlantiff = True if winningParty == plantiff or winningParty == case["first_party"] else False
        court = case["heard_by"][0]["identifier"]
        votes = {}
        for vote in case["decisions"][0]["votes"]:
            if vote["vote"] != "none":
                if inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "plantiff"
                elif not inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif not inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "plantiff"
        return {
            "plantiff": plantiff,
            "defendant": defendant,
            "question": question,
            "facts": facts,
            "court": court,
            "winningParty": winningParty,
            "inFavorPlantiff": inFavorPlantiff,
            "votes": votes
        }
    except:
        pass

In [6]:
processedCases = []
for case in validCases:
    paresed = parseCase(case)
    if paresed is not None:
        processedCases.append(paresed)

In [7]:
len(processedCases)

2938

In [8]:
processedCases[0]

{'plantiff': 'Calderone',
 'defendant': 'Thompson',
 'question': "Did the Court of Appeal's order recalling its mandate denying Thomas M. defendant all habeas relief violate 28 USC section 2244(b) as amended by the Antiterrorism and Effective Death Penalty Act of 1996? Was the order an abuse of the appellate court's discretion?",
 'facts': "In 1983, Thomas M. defendant was convicted of the rape and murder of Ginger Fleischli in California state court. The special circumstance found by the jury of murder during the commission of rape made defendant eligible for the death penalty. In 1995, a federal District Court invalidated defendant's death sentence by granted relief on his rape conviction and the rape special circumstance. In reversing, the Court of Appeals reinstated defendant's death sentence, noting that the State presented strong evidence of rape at trial. The Court of Appeals then issued a mandate denying all habeas relief. Two days before defendant's execution, the Court of App

In [9]:
import pickle

with open('processedCases.pickle', 'wb') as handle:
    pickle.dump(processedCases, handle, protocol=pickle.HIGHEST_PROTOCOL)